# Development Pipeline

## Setup

In [64]:
import boto3
import pandas as pd
import numpy as np
import time
import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup

In [65]:
%env AWS_PROFILE=aeroxye-sagemaker

env: AWS_PROFILE=aeroxye-sagemaker


In [66]:
!aws sts get-caller-identity

{
    "UserId": "AROAWC4YSIQL5OBFCNGEX:botocore-session-1687435302",
    "Account": "418542404631",
    "Arn": "arn:aws:sts::418542404631:assumed-role/SageMaker-UserRole/botocore-session-1687435302"
}


In [67]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SageMaker-UserRole')['Role']['Arn']

region = boto3.Session().region_name
print(f'Current region: {region}')

boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
sagemaker_client.list_feature_groups()

featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Current region: ap-southeast-1


In [68]:
from sagemaker.workflow.pipeline_context import PipelineSession
pipeline_session = PipelineSession()

model_type = 'mostpop'
script_name = 'train.py'

pipeline_name = f"petfinder6000-dev-pipeline-{model_type}"  # SageMaker Pipeline name
model_package_group_name = f"PetFinder6000-{model_type}"
default_model_approval_status = "PendingManualApproval"

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


## Define pipeline parameters

In [69]:
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat

default_bucket = sagemaker_session.default_bucket()
image_uri = '418542404631.dkr.ecr.ap-southeast-1.amazonaws.com/petfinder6000:cornac-39-v2'
exp_name = model_type
model_name = f'tuning-{model_type}-'
current_time = time.strftime("%Y%m%d-%H-%M-%S", time.gmtime())
run_name = f"{model_name}{current_time}"

# prefixes
job_prefix = f'{pipeline_name}/training'
job_name = f"{job_prefix}/{run_name}"

tuning_path = f"s3://{default_bucket}/{job_name}"

eval_job_name = f"{job_name}/evaluation"
report_path = f"s3://{default_bucket}/{eval_job_name}/report"
evaluation_s3_uri = f"{report_path}/output/evaluation/evaluation.json"

# processing step parameters
processing_instance_type = ParameterString(name="ProcessingInstanceType", default_value="ml.m5.xlarge")
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

# training step parameters
training_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
training_instance_count = ParameterInteger(name="TrainingInstanceCount", default_value=1)
max_hyperparam_jobs = ParameterInteger(name="MaximumHyperParamTuningJobs", default_value=3)
max_parallel_hyperparam_jobs = ParameterInteger(name="MaxParallelHyperparamTuningJobs", default_value=3)
hyperparam_tuning_strategy = ParameterString(name="HyperparamStrategy", default_value="Random")

# setup hyperparameters
k = ParameterInteger(name="TrainHyperParamK", default_value=50)
max_iter = ParameterInteger(name="TrainHyperParamMaxIter", default_value=200)
learning_rate = ParameterFloat(name="TrainHyperParamLearningRate", default_value=0.001)
lambda_reg = ParameterFloat(name="TrainHyperParamLambdaReg", default_value=0.001)

# inference step parameters
inference_instance_type = ParameterString(name="InferenceInstanceType", default_value="ml.m5.xlarge")
inference_image = ParameterString(name="InferenceImageURI", default_value='418542404631.dkr.ecr.ap-southeast-1.amazonaws.com/petfinder6000:cornac-39-inference-v15')

# model performance step parameters
harmonic_mean_threshold = ParameterFloat(name="HarmonicMeanThreshold", default_value=0.001)

## Get latest data

In [70]:
strat_bucket = "petfinder6000"
strat_path = "data/training/strat/"

In [71]:
from sagemaker.lambda_helper import Lambda

# Lambda helper class can be used to create the Lambda function
func = Lambda(
    function_name="fetchLatestData",
    execution_role_arn=role,
    script="load_data/get_latest_data.py",
    handler="get_latest_data.handler",
    timeout=600,
    memory_size=128,
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [72]:
from sagemaker.workflow.lambda_step import (LambdaStep,LambdaOutput,LambdaOutputTypeEnum)

step_latest_data_fetch = LambdaStep(
    name="FetchLatestData",
    lambda_func=func,
    inputs={
        "bucket": strat_bucket,
        "object_path": strat_path
    },
    outputs=[
        LambdaOutput(output_name="TrainUri", output_type=LambdaOutputTypeEnum.String),
        LambdaOutput(output_name="EvalUri", output_type=LambdaOutputTypeEnum.String),
        LambdaOutput(output_name="TestUri", output_type=LambdaOutputTypeEnum.String),
    ],
)

## Creating tuning step

In [73]:
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput

estimator = TensorFlow(
    image_uri=image_uri,
    entry_point=script_name,
    dependencies=[
        './training/metrics/harmonic_mean.py',
        './training/metrics/combined_eval_method.py',
        './training/metrics/serendipity_wrapper.py'
    ],
    role=role,
    sagemaker_session=pipeline_session,
    instance_count=training_instance_count,
    instance_type=training_instance_type,
    output_path=tuning_path,
    source_dir="./training",
    environment={"REGION": region},
    hyperparameters={
        "k": k,
        "max_iter": max_iter,
        "learning_rate": learning_rate,
        "lambda_reg": lambda_reg,
    },
)

# configure hyperparameter tuning
hyperparameter_ranges = {
    "k": IntegerParameter(10, 500),
    "max_iter": IntegerParameter(50, 200),
    "learning_rate": ContinuousParameter(0.001, 0.1),
    "lambda_reg": ContinuousParameter(0.001, 0.1),
}
objective_metric_name = "HarmonicMean"
objective_type = "Maximize"
metric_definitions = [{"Name": "HarmonicMean", "Regex": "HarmonicMean: ([0-9\\.]+)"},
                      {"Name": "Serendipity", "Regex": "Serendipity: ([0-9\\.]+)"},
                      {"Name": "F1@10", "Regex": "F1@10: ([0-9\\.]+)"},
                      {"Name": "NDCG@-1", "Regex": "NDCG@-1: ([0-9\\.]+)"},
                      {"Name": "NCRR@-1", "Regex": "NCRR@-1: ([0-9\\.]+)"}]
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=max_hyperparam_jobs,
    max_parallel_jobs=max_parallel_hyperparam_jobs,
    strategy=hyperparam_tuning_strategy,
)

# tune hyperparameter
train_args = tuner.fit(
    inputs={
        "train": TrainingInput(s3_data=step_latest_data_fetch.properties.Outputs["TrainUri"], content_type="text/csv"),
        "eval": TrainingInput(s3_data=step_latest_data_fetch.properties.Outputs["EvalUri"], content_type="text/csv"),
    },
    job_name=job_name,
    include_cls_metadata=False,
    wait=False
)

C:\Users\yongr\anaconda3\envs\cornac39\lib\site-packages\sagemaker\workflow\pipeline_context.py:286: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [74]:
from sagemaker.workflow.steps import TuningStep

step_tuning = TuningStep(name="TrainAndTuneModel", step_args=train_args)

## Get best model

In [75]:
from sagemaker.workflow.model_step import ModelStep

from sagemaker.tensorflow import TensorFlowModel
from sagemaker.model_metrics import MetricsSource, ModelMetrics

best_model_path = step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=default_bucket, prefix=job_name)

best_model = TensorFlowModel(
    image_uri=inference_image,
    source_dir="./inference",
    model_data=best_model_path,
    role=role,
    sagemaker_session=pipeline_session
)

## Evaluate model

In [76]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.processing import FrameworkProcessor, ProcessingInput, ProcessingOutput
from sagemaker.sklearn import SKLearn

evaluate_model_processor = FrameworkProcessor(
    role=role,
    image_uri=image_uri,
    estimator_cls=SKLearn,
    framework_version='0.23-1',
    command=["python3"],
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
    sagemaker_session=pipeline_session,
)

evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

eval_args = evaluate_model_processor.run(
    inputs=[
        ProcessingInput(source=best_model_path, destination="/opt/ml/processing/model"),
        ProcessingInput(source=step_latest_data_fetch.properties.Outputs["TrainUri"], destination="/opt/ml/processing/train"),
        ProcessingInput(source=step_latest_data_fetch.properties.Outputs["TestUri"], destination="/opt/ml/processing/test"),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination=report_path),
    ],
    code="evaluate.py",
    source_dir="./evaluation",
    dependencies=[
        './evaluation/metrics/harmonic_mean.py',
        './evaluation/metrics/combined_eval_method.py',
        './evaluation/metrics/serendipity_wrapper.py'
    ],
    job_name=eval_job_name,
)

In [77]:
from sagemaker.workflow.steps import ProcessingStep

step_evaluate_model = ProcessingStep(
    name="EvaluateModelPerformance",
    step_args=eval_args,
    property_files=[evaluation_report],
)

## Register model

In [78]:
from sagemaker import PipelineModel

pipeline_model = PipelineModel(
    models=[best_model], role=role, sagemaker_session=pipeline_session
)

In [79]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=evaluation_s3_uri,
        content_type="application/json",
    )
)

register_args = pipeline_model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=[inference_instance_type],
    transform_instances=[inference_instance_type],
    model_metrics=model_metrics,
    model_package_group_name=model_package_group_name,
    approval_status=default_model_approval_status,
)

In [80]:
step_register_pipeline_model = ModelStep(
    name="PipelineModel",
    step_args=register_args,
)

## Conditional step

In [81]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

# Create accuracy condition to ensure the model meets performance requirements.
# Models with a test accuracy lower than the condition will not be registered with the model registry.
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_evaluate_model.name,
        property_file=evaluation_report,
        json_path="ranking_metrics.harmonicmean.value",
    ),
    right=harmonic_mean_threshold,
)

# Create a Sagemaker Pipelines ConditionStep, using the condition above.
# Enter the steps to perform if the condition returns True / False.
step_cond = ConditionStep(
    name="HarmonicMean-Greater-Than-Threshold-Condition",
    conditions=[cond_gte],
    if_steps=[step_register_pipeline_model],  # step_register_model, step_register_scaler,
    else_steps=[],
)

## Create pipeline

In [82]:
from sagemaker.workflow.pipeline import Pipeline

# Create a Sagemaker Pipeline.
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        training_instance_count,
        max_hyperparam_jobs,
        max_parallel_hyperparam_jobs,
        hyperparam_tuning_strategy,
        k,
        max_iter,
        learning_rate,
        lambda_reg,
        inference_instance_type,
        inference_image,
        harmonic_mean_threshold
    ],
    steps=[step_latest_data_fetch, step_tuning, step_evaluate_model, step_cond],
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [83]:
pipeline.upsert(role_arn=role)

Using provided s3_resource


INFO:sagemaker.processing:Uploaded ./evaluation to s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-dev-pipeline-mostpop/code/c6c681627f1e2d11032438807c243756/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-dev-pipeline-mostpop/code/10391177712b105adca4cb83b613ba60/runproc.sh


Using provided s3_resource


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:Uploaded ./evaluation to s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-dev-pipeline-mostpop/code/c6c681627f1e2d11032438807c243756/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-1-418542404631/petfinder6000-dev-pipeline-mostpop/code/10391177712b105adca4cb83b613ba60/runproc.sh


{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-1:418542404631:pipeline/petfinder6000-dev-pipeline-mostpop',
 'ResponseMetadata': {'RequestId': '556e9fab-b84d-4084-ad67-29646776a172',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '556e9fab-b84d-4084-ad67-29646776a172',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '107',
   'date': 'Thu, 22 Jun 2023 12:19:27 GMT'},
  'RetryAttempts': 0}}

In [84]:
execution = pipeline.start()

In [44]:
execution.wait()